# Dockerfile directives

Here is a closer look on different docker directives with examples.

## Execute with run

There are two dockerfile directives that allow to execute something with start of docker container: `CMD` and `ENTRYPOINT`.

---

Here is example of the image that modifies alpine in way to print message `Hello world` during container start.

In [ ]:
cat << EOF > dockerfile
FROM alpine
CMD ["echo", "'Hello world'"]
EOF

docker build -t temp_image . &> /dev/null
docker run --rm temp_image

docker rmi temp_image &> /dev/null
rm dockerfile

'Hello world'


## Parametrisation

With the `ENV` and `ARG` directives, you can define values to be reused during different phases of the build. There are some differences between the two; refer to the [Docker documentation page](https://docs.docker.com/build/building/variables/).

### Environment variables

For creating environment variable you can use `ENV` dockerfile instruction. Feature of this diretive is that defined variable will be included to the environment of the resulting containers.

---

In the following example, a Dockerfile is created using `ENV` to define `TEST_VAR` with the value `"env_variable"`, and a container is started based on the resulting image.

In [27]:
cat << EOF > /tmp/dockerfile
FROM alpine
ENV TEST_VAR="env_variable"
WORKDIR \$TEST_VAR
EOF

docker build -t test_image -f /tmp/dockerfile . &> /dev/null
docker run -itd --name env_example --rm test_image &> /dev/null

Variable used in `dockerfile` as name for new directory. The following cell checks if the variable was actually created.

In [28]:
docker exec env_example pwd

/env_variable


But you can still use variables from the container. The following cell demonstrates this by showing the `env` of the container in question. 

In [ ]:
docker exec env_example env

PATH=/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
HOSTNAME=a2b1510d2ee2
TEST_VAR=env_variable
HOME=/root


The following cell clears the environment after experiments.

In [30]:
docker stop env_example &> /dev/null
docker rmi test_image &> /dev/null

### Build arguments

Build arguments that can be created with `ARG` directive in nginx, it is a variable that can be used for build command for dockerfile but there is no information about it after build.

---

The following cell builds image and runs container based on it, by properties of gotten container we'll learn properties of the variable defined with `ARG` directive.

In [32]:
cat << EOF > /tmp/dockerfile
FROM alpine
ARG TEST_VAR="arg_variable"
WORKDIR \$TEST_VAR
EOF

docker build -t test_image -f /tmp/dockerfile . &> /dev/null
docker run -itd --name arg_example --rm test_image &> /dev/null

The variable is used as a directory name for the `WORKDIR` directive, as a result the initial folder of the container is the same as the value of the variable.

In [33]:
docker exec arg_example pwd

/arg_variable


But there are no such variables in the environment of the resulting container, which is shown by the following cell.

In [34]:
docker exec arg_example env

PATH=/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
HOSTNAME=2123ad7acb59
HOME=/root


Don't forget to clean up the environment.

In [35]:
docker stop arg_example &> /dev/null
docker rmi test_image &> /dev/null

#### Specify in build

You can specify the argument value during the build with the `--build-arg <arg name>=<value>` argument of to the `docker build`.

---

The following example builds a `dockerfile` which working directory depends on the `TEST_VAR` argument, and builds the corresponding image with `--build-args TEST_VAR=name_for_folder`.

In [5]:
cat << EOF > /tmp/dockerfile
FROM alpine
ARG TEST_VAR
WORKDIR \$TEST_VAR
EOF

docker build --build-arg TEST_VAR=name_for_folder -t test_image -f /tmp/dockerfile . &> /dev/null

The following cell shows the `WORKDIR` of the resulting image corresponding to the value of the `TEST_VAR`.

In [6]:
docker run --rm test_image pwd

/name_for_folder


Cleaning the environment.

In [7]:
docker rmi test_image &> /dev/null